In [23]:
# ===== REQUIRED IMPORTS =====

import cv2
import numpy as np
from scipy.signal import find_peaks
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import threading
import queue
import time
from collections import deque

print("✅ All imports loaded")
print("System: Advanced LED Detection for 2Hz DC LEDs (10-120m range)")
print("Features: Adaptive preprocessing, FFT analysis, multi-range color detection, flicker tracking, temporal confirmation")

✅ All imports loaded
System: Advanced LED Detection for 2Hz DC LEDs (10-120m range)
Features: Adaptive preprocessing, FFT analysis, multi-range color detection, flicker tracking, temporal confirmation


In [24]:
# ===== ADVANCED LED DETECTION SYSTEM =====
# Multi-stage detection: Adaptive preprocessing + Frequency + Color + Flicker
# Optimized for 2Hz DC-powered LEDs at 10-120m distance

import numpy as np
import cv2
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import threading
import queue
import time
from collections import deque
from scipy.signal import find_peaks
from scipy import ndimage
import logging

logging.basicConfig(level=logging.INFO)

# ===== SYSTEM PARAMETERS =====

class DetectionParams:
    """Centralized parameter management"""
    def __init__(self):
        # General
        self.LED_FREQ_TARGET = 2.0  # Target LED frequency (Hz)
        self.FREQ_TOLERANCE = 0.5   # Frequency tolerance
        self.FPS = 30.0             # Camera FPS
        
        # Grid settings
        self.GRID_SIZE = 80         # Grid cell size (pixels)
        self.MIN_GRID_SIZE = 40
        self.MAX_GRID_SIZE = 200
        
        # Adaptive thresholding
        self.ADAPTIVE_METHOD = 'gaussian'  # 'gaussian' or 'mean'
        self.ADAPTIVE_BLOCK_SIZE = 11      # Must be odd
        self.ADAPTIVE_C = 2                 # Constant subtracted
        
        # Frequency analysis
        self.FREQ_BUFFER_SIZE = 60   # Frames to analyze (2 sec at 30fps)
        self.MIN_FREQ = 0.5          # Minimum detectable frequency
        self.MAX_FREQ = 10.0         # Maximum detectable frequency
        self.FFT_THRESHOLD = 3.0     # Peak detection threshold
        
        # HSV Color ranges (multiple for robustness)
        self.HSV_RANGES = [
            # Range 1: Bright yellow
            ([20, 100, 150], [35, 255, 255]),
            # Range 2: Orange-yellow
            ([15, 80, 120], [40, 255, 255]),
            # Range 3: Dim yellow (shadow)
            ([18, 50, 80], [38, 255, 200])
        ]
        
        # LAB Color space (better for illumination)
        self.LAB_L_MIN = 100         # Minimum lightness
        self.LAB_A_RANGE = (120, 135) # Yellow in A channel
        self.LAB_B_RANGE = (135, 160) # Yellow in B channel
        
        # Flicker detection
        self.FLICKER_BUFFER_SIZE = 30  # Frames for flicker analysis
        self.FLICKER_STD_THRESHOLD = 15.0  # Std dev threshold
        self.INTENSITY_CHANGE_THRESHOLD = 20  # Min intensity change
        
        # Size filtering (distance-adaptive)
        self.MIN_AREA = 25           # Minimum LED area (pixels)
        self.MAX_AREA = 10000        # Maximum LED area (pixels)
        self.ASPECT_RATIO_MAX = 3.0  # Width/height ratio limit
        
        # Temporal tracking
        self.TEMPORAL_BUFFER_SIZE = 10  # Frames to confirm detection
        self.MIN_DETECTIONS = 5         # Minimum detections to confirm
        self.POSITION_TOLERANCE = 20    # Max position change (pixels)
        
        # Confidence scoring weights
        self.WEIGHT_FREQUENCY = 0.35
        self.WEIGHT_COLOR = 0.25
        self.WEIGHT_FLICKER = 0.25
        self.WEIGHT_TEMPORAL = 0.15
        self.CONFIDENCE_THRESHOLD = 0.5  # Minimum confidence to report

params = DetectionParams()
print("✅ Detection parameters initialized")
print(f"🎯 Target: {params.LED_FREQ_TARGET} Hz LEDs at 10-120m range")

✅ Detection parameters initialized
🎯 Target: 2.0 Hz LEDs at 10-120m range


In [25]:
# ===== STAGE 1: ADAPTIVE PREPROCESSING =====

class AdaptivePreprocessor:
    """Handles varying lighting conditions with adaptive methods"""
    
    def __init__(self, params):
        self.params = params
        
    def preprocess_frame(self, frame):
        """Apply adaptive preprocessing to handle lighting variations"""
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)
        
        # Adaptive thresholding
        if self.params.ADAPTIVE_METHOD == 'gaussian':
            binary = cv2.adaptiveThreshold(
                enhanced, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY, self.params.ADAPTIVE_BLOCK_SIZE, 
                self.params.ADAPTIVE_C
            )
        else:
            binary = cv2.adaptiveThreshold(
                enhanced, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                cv2.THRESH_BINARY, self.params.ADAPTIVE_BLOCK_SIZE,
                self.params.ADAPTIVE_C
            )
        
        return gray, enhanced, binary
    
    def get_local_statistics(self, gray, grid_size):
        """Calculate local mean and std for each grid cell"""
        h, w = gray.shape
        grid_h = h // grid_size
        grid_w = w // grid_size
        
        stats = []
        for i in range(grid_h):
            for j in range(grid_w):
                y1, y2 = i * grid_size, min((i + 1) * grid_size, h)
                x1, x2 = j * grid_size, min((j + 1) * grid_size, w)
                roi = gray[y1:y2, x1:x2]
                
                stats.append({
                    'position': (j, i),
                    'bbox': (x1, y1, x2, y2),
                    'mean': np.mean(roi),
                    'std': np.std(roi),
                    'max': np.max(roi),
                    'min': np.min(roi)
                })
        
        return stats

preprocessor = AdaptivePreprocessor(params)
print("✅ Adaptive preprocessor initialized")

✅ Adaptive preprocessor initialized


In [26]:
# ===== STAGE 2: FREQUENCY ANALYZER =====

class FrequencyAnalyzer:
    """FFT-based frequency detection for ALL frequencies"""
    
    def __init__(self, params):
        self.params = params
        self.buffers = {}  # Store intensity buffer per grid cell
        
    def add_intensity(self, grid_id, intensity):
        """Add intensity value to grid cell buffer"""
        if grid_id not in self.buffers:
            self.buffers[grid_id] = deque(maxlen=self.params.FREQ_BUFFER_SIZE)
        self.buffers[grid_id].append(intensity)
    
    def analyze_frequency(self, grid_id):
        """Perform FFT and return ALL detected frequencies"""
        if grid_id not in self.buffers or len(self.buffers[grid_id]) < 30:
            return []
        
        signal = np.array(self.buffers[grid_id])
        
        # Remove DC component
        signal = signal - np.mean(signal)
        
        if np.std(signal) < 1e-6:
            return []
        
        # Apply FFT
        fft_result = np.fft.rfft(signal)
        fft_magnitude = np.abs(fft_result)[1:]  # Skip DC
        
        # Frequency axis
        freq_axis = np.fft.rfftfreq(len(signal), 1.0 / self.params.FPS)[1:]
        
        # Find all peaks
        threshold = np.max(fft_magnitude) * (self.params.FFT_THRESHOLD / 10.0)
        peaks, properties = find_peaks(fft_magnitude, height=threshold)
        
        # Extract frequencies with confidence
        detected_freqs = []
        for peak_idx in peaks:
            if 0 <= peak_idx < len(freq_axis):
                freq = freq_axis[peak_idx]
                if self.params.MIN_FREQ <= freq <= self.params.MAX_FREQ:
                    magnitude = fft_magnitude[peak_idx]
                    confidence = min(magnitude / (np.max(fft_magnitude) + 1e-6), 1.0)
                    
                    # Check if close to target frequency
                    is_target = abs(freq - self.params.LED_FREQ_TARGET) < self.params.FREQ_TOLERANCE
                    
                    detected_freqs.append({
                        'frequency': float(freq),
                        'magnitude': float(magnitude),
                        'confidence': float(confidence),
                        'is_target': is_target
                    })
        
        # Sort by magnitude (strongest first)
        detected_freqs.sort(key=lambda x: x['magnitude'], reverse=True)
        return detected_freqs
    
    def get_target_frequency_score(self, frequencies):
        """Calculate confidence score for target frequency (2Hz)"""
        for freq_info in frequencies:
            if freq_info['is_target']:
                return freq_info['confidence']
        return 0.0

freq_analyzer = FrequencyAnalyzer(params)
print("✅ Frequency analyzer initialized - detects ALL frequencies")

✅ Frequency analyzer initialized - detects ALL frequencies


In [27]:
# ===== STAGE 3: MULTI-HSV COLOR DETECTOR =====

class ColorDetector:
    """Multi-range HSV + LAB color detection for robustness"""
    
    def __init__(self, params):
        self.params = params
    
    def detect_by_color(self, frame):
        """Detect yellow objects using multiple HSV ranges + LAB"""
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        
        # Combine all HSV ranges
        combined_mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
        
        for lower, upper in self.params.HSV_RANGES:
            lower_np = np.array(lower, dtype=np.uint8)
            upper_np = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(hsv, lower_np, upper_np)
            combined_mask = cv2.bitwise_or(combined_mask, mask)
        
        # LAB color space filtering
        l_channel, a_channel, b_channel = cv2.split(lab)
        lab_mask = cv2.inRange(l_channel, self.params.LAB_L_MIN, 255)
        lab_mask = cv2.bitwise_and(lab_mask, 
                                    cv2.inRange(a_channel, self.params.LAB_A_RANGE[0], self.params.LAB_A_RANGE[1]))
        lab_mask = cv2.bitwise_and(lab_mask,
                                    cv2.inRange(b_channel, self.params.LAB_B_RANGE[0], self.params.LAB_B_RANGE[1]))
        
        # Combine HSV and LAB masks
        final_mask = cv2.bitwise_or(combined_mask, lab_mask)
        
        # Morphological operations to clean up
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_OPEN, kernel)
        final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter by size and aspect ratio
        candidates = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if self.params.MIN_AREA <= area <= self.params.MAX_AREA:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h) if h > 0 else 0
                
                if aspect_ratio <= self.params.ASPECT_RATIO_MAX:
                    # Calculate color confidence (how much of object is in color range)
                    roi_mask = final_mask[y:y+h, x:x+w]
                    color_confidence = np.sum(roi_mask > 0) / (w * h * 255.0)
                    
                    candidates.append({
                        'bbox': (x, y, w, h),
                        'area': area,
                        'center': (x + w // 2, y + h // 2),
                        'color_confidence': color_confidence,
                        'contour': contour
                    })
        
        return candidates, final_mask

color_detector = ColorDetector(params)
print("✅ Multi-range color detector initialized (HSV + LAB)")

✅ Multi-range color detector initialized (HSV + LAB)


In [28]:
# ===== STAGE 4: FLICKER DETECTOR =====

class FlickerDetector:
    """Temporal intensity analysis to detect flickering"""
    
    def __init__(self, params):
        self.params = params
        self.region_buffers = {}  # Store intensity history per region
    
    def add_region_intensity(self, region_id, mean_intensity):
        """Add intensity measurement for a region"""
        if region_id not in self.region_buffers:
            self.region_buffers[region_id] = deque(maxlen=self.params.FLICKER_BUFFER_SIZE)
        self.region_buffers[region_id].append(mean_intensity)
    
    def analyze_flicker(self, region_id):
        """Analyze if region shows flickering behavior"""
        if region_id not in self.region_buffers or len(self.region_buffers[region_id]) < 10:
            return {'is_flickering': False, 'confidence': 0.0, 'std': 0.0}
        
        intensities = np.array(self.region_buffers[region_id])
        
        # Calculate statistics
        std_dev = np.std(intensities)
        mean_intensity = np.mean(intensities)
        intensity_range = np.max(intensities) - np.min(intensities)
        
        # Flickering indicators
        is_flickering = (std_dev > self.params.FLICKER_STD_THRESHOLD and 
                        intensity_range > self.params.INTENSITY_CHANGE_THRESHOLD)
        
        # Calculate confidence based on std deviation
        confidence = min(std_dev / 50.0, 1.0) if is_flickering else 0.0
        
        # Check periodicity (optional: detect if flicker is periodic)
        periodicity_score = 0.0
        if len(intensities) >= 20:
            # Simple autocorrelation check
            normalized = (intensities - mean_intensity) / (std_dev + 1e-6)
            autocorr = np.correlate(normalized, normalized, mode='same')
            autocorr = autocorr[len(autocorr)//2:]
            
            # Look for peaks in autocorrelation (indicates periodicity)
            if len(autocorr) > 5:
                peaks, _ = find_peaks(autocorr[1:], height=0.3)
                if len(peaks) > 0:
                    periodicity_score = 0.5
        
        return {
            'is_flickering': is_flickering,
            'confidence': confidence + periodicity_score,
            'std': std_dev,
            'range': intensity_range,
            'mean': mean_intensity
        }
    
    def clear_old_regions(self, active_region_ids):
        """Remove buffers for regions no longer being tracked"""
        to_remove = [rid for rid in self.region_buffers.keys() if rid not in active_region_ids]
        for rid in to_remove:
            del self.region_buffers[rid]

flicker_detector = FlickerDetector(params)
print("✅ Flicker detector initialized")

✅ Flicker detector initialized


In [29]:
# ===== STAGE 5: TEMPORAL TRACKER & CONFIDENCE SCORER =====

class TemporalTracker:
    """Track detections over time to confirm LEDs"""
    
    def __init__(self, params):
        self.params = params
        self.tracked_objects = {}  # {id: detection_history}
        self.next_id = 0
    
    def update(self, current_detections):
        """Update tracking with current frame detections"""
        # Match current detections with tracked objects
        matched = {}
        unmatched_current = list(range(len(current_detections)))
        
        for obj_id, history in self.tracked_objects.items():
            if len(history) == 0:
                continue
            
            last_det = history[-1]
            last_pos = last_det['center']
            
            # Find closest unmatched detection
            best_match = None
            best_dist = self.params.POSITION_TOLERANCE
            
            for idx in unmatched_current:
                curr_pos = current_detections[idx]['center']
                dist = np.sqrt((curr_pos[0] - last_pos[0])**2 + (curr_pos[1] - last_pos[1])**2)
                
                if dist < best_dist:
                    best_dist = dist
                    best_match = idx
            
            if best_match is not None:
                matched[obj_id] = best_match
                unmatched_current.remove(best_match)
        
        # Update matched objects
        new_tracked = {}
        for obj_id, det_idx in matched.items():
            detection = current_detections[det_idx].copy()
            detection['timestamp'] = time.time()
            
            history = self.tracked_objects[obj_id]
            history.append(detection)
            if len(history) > self.params.TEMPORAL_BUFFER_SIZE:
                history.pop(0)
            new_tracked[obj_id] = history
        
        # Add new objects
        for idx in unmatched_current:
            detection = current_detections[idx].copy()
            detection['timestamp'] = time.time()
            new_tracked[self.next_id] = [detection]
            self.next_id += 1
        
        # Remove stale objects (not seen for 2+ seconds)
        current_time = time.time()
        self.tracked_objects = {
            obj_id: history for obj_id, history in new_tracked.items()
            if current_time - history[-1]['timestamp'] < 2.0
        }
        
        return self.tracked_objects
    
    def get_confirmed_objects(self):
        """Return objects confirmed over multiple frames"""
        confirmed = []
        for obj_id, history in self.tracked_objects.items():
            if len(history) >= self.params.MIN_DETECTIONS:
                # Calculate average position and confidence
                avg_conf = np.mean([d.get('total_confidence', 0) for d in history])
                last_det = history[-1]
                
                confirmed.append({
                    'id': obj_id,
                    'position': last_det['center'],
                    'bbox': last_det['bbox'],
                    'confidence': avg_conf,
                    'detections': len(history),
                    'stable': len(history) == self.params.TEMPORAL_BUFFER_SIZE
                })
        
        return confirmed


class ConfidenceScorer:
    """Combine all detection methods into final confidence score"""
    
    def __init__(self, params):
        self.params = params
    
    def calculate_confidence(self, detection):
        """Calculate weighted confidence from all methods"""
        # Frequency score (from FFT)
        freq_score = detection.get('frequency_confidence', 0.0)
        
        # Color score (from HSV/LAB)
        color_score = detection.get('color_confidence', 0.0)
        
        # Flicker score (from temporal analysis)
        flicker_score = detection.get('flicker_confidence', 0.0)
        
        # Temporal score (from tracking history)
        temporal_score = detection.get('temporal_confidence', 0.0)
        
        # Weighted combination
        total_confidence = (
            freq_score * self.params.WEIGHT_FREQUENCY +
            color_score * self.params.WEIGHT_COLOR +
            flicker_score * self.params.WEIGHT_FLICKER +
            temporal_score * self.params.WEIGHT_TEMPORAL
        )
        
        return total_confidence, {
            'frequency': freq_score,
            'color': color_score,
            'flicker': flicker_score,
            'temporal': temporal_score,
            'total': total_confidence
        }

temporal_tracker = TemporalTracker(params)
confidence_scorer = ConfidenceScorer(params)
print("✅ Temporal tracker and confidence scorer initialized")

✅ Temporal tracker and confidence scorer initialized


In [30]:
# ===== INTEGRATED DETECTION ENGINE =====

class AdvancedLEDDetector:
    """Main detection engine combining all stages"""
    
    def __init__(self, params):
        self.params = params
        self.preprocessor = AdaptivePreprocessor(params)
        self.freq_analyzer = FrequencyAnalyzer(params)
        self.color_detector = ColorDetector(params)
        self.flicker_detector = FlickerDetector(params)
        self.temporal_tracker = TemporalTracker(params)
        self.confidence_scorer = ConfidenceScorer(params)
        
        self.frame_count = 0
        self.grid_stats = []
        
    def process_frame(self, frame):
        """Full detection pipeline on single frame"""
        self.frame_count += 1
        h, w = frame.shape[:2]
        
        # Stage 1: Adaptive preprocessing
        gray, enhanced, binary = self.preprocessor.preprocess_frame(frame)
        self.grid_stats = self.preprocessor.get_local_statistics(gray, self.params.GRID_SIZE)
        
        # Stage 2: Update frequency buffers for all grid cells
        for stat in self.grid_stats:
            grid_id = f"{stat['position'][0]}_{stat['position'][1]}"
            self.freq_analyzer.add_intensity(grid_id, stat['mean'])
        
        # Stage 3: Color-based detection
        color_candidates, color_mask = self.color_detector.detect_by_color(frame)
        
        # Stage 4: Combine with flicker analysis
        detections = []
        active_regions = []
        
        for candidate in color_candidates:
            x, y, w, h = candidate['bbox']
            center = candidate['center']
            
            # Get intensity from gray frame
            roi = gray[y:y+h, x:x+w]
            mean_intensity = np.mean(roi)
            
            # Create region ID
            region_id = f"reg_{center[0]}_{center[1]}"
            active_regions.append(region_id)
            
            # Update flicker detector
            self.flicker_detector.add_region_intensity(region_id, mean_intensity)
            flicker_result = self.flicker_detector.analyze_flicker(region_id)
            
            # Check frequency for grid cell containing this candidate
            grid_x = center[0] // self.params.GRID_SIZE
            grid_y = center[1] // self.params.GRID_SIZE
            grid_id = f"{grid_x}_{grid_y}"
            frequencies = self.freq_analyzer.analyze_frequency(grid_id)
            freq_confidence = self.freq_analyzer.get_target_frequency_score(frequencies)
            
            # Build detection dict
            detection = {
                'bbox': candidate['bbox'],
                'center': center,
                'area': candidate['area'],
                'color_confidence': candidate['color_confidence'],
                'frequency_confidence': freq_confidence,
                'flicker_confidence': flicker_result['confidence'],
                'frequencies': frequencies[:3],  # Top 3 frequencies
                'flicker_std': flicker_result['std']
            }
            
            detections.append(detection)
        
        # Clean up old flicker regions
        self.flicker_detector.clear_old_regions(active_regions)
        
        # Stage 5: Temporal tracking
        tracked = self.temporal_tracker.update(detections)
        
        # Calculate final confidences
        final_detections = []
        for detection in detections:
            total_conf, breakdown = self.confidence_scorer.calculate_confidence(detection)
            detection['total_confidence'] = total_conf
            detection['confidence_breakdown'] = breakdown
            
            if total_conf >= self.params.CONFIDENCE_THRESHOLD:
                final_detections.append(detection)
        
        # Get confirmed objects
        confirmed_leds = self.temporal_tracker.get_confirmed_objects()
        
        return {
            'frame': frame,
            'gray': gray,
            'enhanced': enhanced,
            'binary': binary,
            'color_mask': color_mask,
            'grid_stats': self.grid_stats,
            'detections': final_detections,
            'confirmed_leds': confirmed_leds,
            'frame_count': self.frame_count
        }
    
    def visualize_results(self, results, show_grid=True, show_frequencies=True, show_confidence=True):
        """Create visualization of detection results"""
        frame = results['frame'].copy()
        h, w = frame.shape[:2]
        
        # Draw grid (optional)
        if show_grid:
            for stat in results['grid_stats']:
                x1, y1, x2, y2 = stat['bbox']
                # Color code by intensity
                if stat['mean'] > 200:
                    color = (0, 255, 255)  # Yellow: bright
                elif stat['mean'] > 150:
                    color = (0, 165, 255)  # Orange: medium
                else:
                    color = (200, 200, 200)  # Gray: dim
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 1)
        
        # Draw detections
        for det in results['detections']:
            x, y, w, h = det['bbox']
            conf = det['total_confidence']
            
            # Color by confidence
            if conf > 0.7:
                box_color = (0, 255, 0)  # Green: high confidence
            elif conf > 0.5:
                box_color = (0, 255, 255)  # Yellow: medium
            else:
                box_color = (0, 165, 255)  # Orange: low
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
            
            # Show confidence breakdown
            if show_confidence:
                breakdown = det['confidence_breakdown']
                text = f"C:{conf:.2f} F:{breakdown['frequency']:.2f} Col:{breakdown['color']:.2f} Fl:{breakdown['flicker']:.2f}"
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, box_color, 1)
            
            # Show frequencies
            if show_frequencies and len(det['frequencies']) > 0:
                freq_text = ", ".join([f"{f['frequency']:.1f}Hz" for f in det['frequencies'][:2]])
                cv2.putText(frame, freq_text, (x, y+h+15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        # Draw confirmed LEDs
        for led in results['confirmed_leds']:
            x, y, w, h = led['bbox']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)  # Red: confirmed
            cv2.putText(frame, f"LED #{led['id']}", (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        # Add summary
        summary = f"Frame: {results['frame_count']} | Detections: {len(results['detections'])} | Confirmed LEDs: {len(results['confirmed_leds'])}"
        cv2.putText(frame, summary, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        return frame

# Create detector instance
detector = AdvancedLEDDetector(params)
print("✅ Advanced LED Detector ready!")
print("📊 Pipeline: Adaptive Preprocessing → Frequency → Color → Flicker → Temporal Tracking")
print(f"🎯 Optimized for {params.LED_FREQ_TARGET}Hz LEDs at 10-120m range")

✅ Advanced LED Detector ready!
📊 Pipeline: Adaptive Preprocessing → Frequency → Color → Flicker → Temporal Tracking
🎯 Optimized for 2.0Hz LEDs at 10-120m range


In [31]:
# ===== ADVANCED PARAMETER CONTROL GUI =====

class AdvancedLEDDetectionGUI:
    """Comprehensive GUI with all parameter controls"""
    
    def __init__(self, master, detector):
        self.master = master
        self.detector = detector
        self.params = detector.params
        
        self.master.title("Advanced LED Detection System - 2Hz DC LEDs (10-120m)")
        self.master.geometry("1600x900")
        
        self.is_running = False
        self.frame_queue = queue.Queue(maxsize=3)
        self.show_grid = tk.BooleanVar(value=True)
        self.show_frequencies = tk.BooleanVar(value=True)
        self.show_confidence = tk.BooleanVar(value=True)
        
        # Video source (for testing - replace with camera)
        self.test_mode = True
        self.test_frame_count = 0
        
        self.create_widgets()
        
    def create_widgets(self):
        """Build main GUI layout"""
        # Main container
        main_container = ttk.Frame(self.master)
        main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Top: Control buttons
        control_frame = ttk.Frame(main_container)
        control_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(control_frame, text="▶ Start Detection", command=self.start_detection, width=18).pack(side=tk.LEFT, padx=5)
        ttk.Button(control_frame, text="⏹ Stop Detection", command=self.stop_detection, width=18).pack(side=tk.LEFT, padx=5)
        ttk.Checkbutton(control_frame, text="Show Grid", variable=self.show_grid).pack(side=tk.LEFT, padx=10)
        ttk.Checkbutton(control_frame, text="Show Frequencies", variable=self.show_frequencies).pack(side=tk.LEFT, padx=10)
        ttk.Checkbutton(control_frame, text="Show Confidence", variable=self.show_confidence).pack(side=tk.LEFT, padx=10)
        
        # Content area
        content_frame = ttk.Frame(main_container)
        content_frame.pack(fill=tk.BOTH, expand=True)
        
        # Left: Video display
        video_frame = ttk.LabelFrame(content_frame, text="Detection View", padding=10)
        video_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 10))
        
        self.video_canvas = tk.Canvas(video_frame, bg='black', width=960, height=720)
        self.video_canvas.pack()
        
        # Status labels
        status_frame = ttk.Frame(video_frame)
        status_frame.pack(fill=tk.X, pady=(10, 0))
        
        self.fps_label = ttk.Label(status_frame, text="FPS: 0.0")
        self.fps_label.pack(side=tk.LEFT, padx=10)
        
        self.detections_label = ttk.Label(status_frame, text="Detections: 0")
        self.detections_label.pack(side=tk.LEFT, padx=10)
        
        self.confirmed_label = ttk.Label(status_frame, text="Confirmed LEDs: 0")
        self.confirmed_label.pack(side=tk.LEFT, padx=10)
        
        # Right: Parameters
        params_frame = ttk.LabelFrame(content_frame, text="Detection Parameters", padding=10)
        params_frame.pack(side=tk.RIGHT, fill=tk.BOTH)
        
        # Create tabbed parameter interface
        notebook = ttk.Notebook(params_frame)
        notebook.pack(fill=tk.BOTH, expand=True)
        
        # Tab 1: Grid & Preprocessing
        tab1 = ttk.Frame(notebook)
        notebook.add(tab1, text="Grid & Preprocessing")
        self.create_preprocessing_controls(tab1)
        
        # Tab 2: Frequency Analysis
        tab2 = ttk.Frame(notebook)
        notebook.add(tab2, text="Frequency Analysis")
        self.create_frequency_controls(tab2)
        
        # Tab 3: Color Detection
        tab3 = ttk.Frame(notebook)
        notebook.add(tab3, text="Color Detection")
        self.create_color_controls(tab3)
        
        # Tab 4: Flicker & Temporal
        tab4 = ttk.Frame(notebook)
        notebook.add(tab4, text="Flicker & Tracking")
        self.create_flicker_controls(tab4)
        
        # Tab 5: Confidence Weights
        tab5 = ttk.Frame(notebook)
        notebook.add(tab5, text="Confidence Scoring")
        self.create_confidence_controls(tab5)
    
    def create_preprocessing_controls(self, parent):
        """Preprocessing parameters"""
        scroll_frame = self.create_scrollable(parent)
        row = 0
        
        ttk.Label(scroll_frame, text="Adaptive Thresholding", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(0,10))
        row += 1
        
        self.add_slider(scroll_frame, "Grid Size", self.params, 'GRID_SIZE', 40, 200, row)
        row += 1
        self.add_slider(scroll_frame, "Block Size (odd)", self.params, 'ADAPTIVE_BLOCK_SIZE', 3, 31, row, step=2)
        row += 1
        self.add_slider(scroll_frame, "Adaptive C", self.params, 'ADAPTIVE_C', -10, 10, row, resolution=0.5)
        row += 1
        
        ttk.Label(scroll_frame, text="Method:", font=('Arial', 9)).grid(row=row, column=0, sticky=tk.W)
        method_var = tk.StringVar(value=self.params.ADAPTIVE_METHOD)
        ttk.Radiobutton(scroll_frame, text="Gaussian", variable=method_var, value='gaussian',
                       command=lambda: setattr(self.params, 'ADAPTIVE_METHOD', method_var.get())).grid(row=row, column=1)
        ttk.Radiobutton(scroll_frame, text="Mean", variable=method_var, value='mean',
                       command=lambda: setattr(self.params, 'ADAPTIVE_METHOD', method_var.get())).grid(row=row, column=2)
    
    def create_frequency_controls(self, parent):
        """Frequency analysis parameters"""
        scroll_frame = self.create_scrollable(parent)
        row = 0
        
        ttk.Label(scroll_frame, text="FFT Configuration", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(0,10))
        row += 1
        
        self.add_slider(scroll_frame, "Target Frequency (Hz)", self.params, 'LED_FREQ_TARGET', 0.5, 10, row, resolution=0.1)
        row += 1
        self.add_slider(scroll_frame, "Frequency Tolerance", self.params, 'FREQ_TOLERANCE', 0.1, 2.0, row, resolution=0.1)
        row += 1
        self.add_slider(scroll_frame, "Buffer Size (frames)", self.params, 'FREQ_BUFFER_SIZE', 30, 120, row)
        row += 1
        self.add_slider(scroll_frame, "FFT Threshold", self.params, 'FFT_THRESHOLD', 1.0, 10.0, row, resolution=0.1)
        row += 1
        self.add_slider(scroll_frame, "Min Frequency", self.params, 'MIN_FREQ', 0.1, 5.0, row, resolution=0.1)
        row += 1
        self.add_slider(scroll_frame, "Max Frequency", self.params, 'MAX_FREQ', 5.0, 20.0, row, resolution=0.1)
    
    def create_color_controls(self, parent):
        """Color detection parameters"""
        scroll_frame = self.create_scrollable(parent)
        row = 0
        
        ttk.Label(scroll_frame, text="HSV Range 1 (Bright Yellow)", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(0,10))
        row += 1
        
        # Simplified: show only first HSV range for adjustment
        ttk.Label(scroll_frame, text="Lower: H", font=('Arial', 8)).grid(row=row, column=0, sticky=tk.W)
        h_lower = tk.IntVar(value=self.params.HSV_RANGES[0][0][0])
        ttk.Scale(scroll_frame, from_=0, to=179, variable=h_lower, orient=tk.HORIZONTAL, length=150,
                 command=lambda v: self.update_hsv_range(0, 0, 0, int(float(v)))).grid(row=row, column=1, columnspan=2)
        row += 1
        
        ttk.Label(scroll_frame, text="LAB Color Space", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(10,10))
        row += 1
        self.add_slider(scroll_frame, "Min Lightness", self.params, 'LAB_L_MIN', 50, 200, row)
        row += 1
        
        ttk.Label(scroll_frame, text="Size Filtering", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(10,10))
        row += 1
        self.add_slider(scroll_frame, "Min Area (pixels)", self.params, 'MIN_AREA', 10, 500, row)
        row += 1
        self.add_slider(scroll_frame, "Max Area (pixels)", self.params, 'MAX_AREA', 1000, 20000, row, step=100)
        row += 1
        self.add_slider(scroll_frame, "Max Aspect Ratio", self.params, 'ASPECT_RATIO_MAX', 1.0, 5.0, row, resolution=0.1)
    
    def create_flicker_controls(self, parent):
        """Flicker and temporal tracking parameters"""
        scroll_frame = self.create_scrollable(parent)
        row = 0
        
        ttk.Label(scroll_frame, text="Flicker Detection", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(0,10))
        row += 1
        
        self.add_slider(scroll_frame, "Flicker Buffer Size", self.params, 'FLICKER_BUFFER_SIZE', 10, 60, row)
        row += 1
        self.add_slider(scroll_frame, "Std Dev Threshold", self.params, 'FLICKER_STD_THRESHOLD', 5.0, 50.0, row, resolution=1.0)
        row += 1
        self.add_slider(scroll_frame, "Intensity Change Threshold", self.params, 'INTENSITY_CHANGE_THRESHOLD', 10, 100, row)
        row += 1
        
        ttk.Label(scroll_frame, text="Temporal Tracking", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(10,10))
        row += 1
        
        self.add_slider(scroll_frame, "Temporal Buffer Size", self.params, 'TEMPORAL_BUFFER_SIZE', 5, 30, row)
        row += 1
        self.add_slider(scroll_frame, "Min Detections to Confirm", self.params, 'MIN_DETECTIONS', 3, 15, row)
        row += 1
        self.add_slider(scroll_frame, "Position Tolerance", self.params, 'POSITION_TOLERANCE', 10, 100, row)
    
    def create_confidence_controls(self, parent):
        """Confidence scoring weights"""
        scroll_frame = self.create_scrollable(parent)
        row = 0
        
        ttk.Label(scroll_frame, text="Method Weights (must sum to 1.0)", font=('Arial', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky=tk.W, pady=(0,10))
        row += 1
        
        self.add_slider(scroll_frame, "Frequency Weight", self.params, 'WEIGHT_FREQUENCY', 0.0, 1.0, row, resolution=0.05)
        row += 1
        self.add_slider(scroll_frame, "Color Weight", self.params, 'WEIGHT_COLOR', 0.0, 1.0, row, resolution=0.05)
        row += 1
        self.add_slider(scroll_frame, "Flicker Weight", self.params, 'WEIGHT_FLICKER', 0.0, 1.0, row, resolution=0.05)
        row += 1
        self.add_slider(scroll_frame, "Temporal Weight", self.params, 'WEIGHT_TEMPORAL', 0.0, 1.0, row, resolution=0.05)
        row += 1
        
        ttk.Separator(scroll_frame, orient='horizontal').grid(row=row, column=0, columnspan=3, sticky='ew', pady=10)
        row += 1
        
        self.add_slider(scroll_frame, "Confidence Threshold", self.params, 'CONFIDENCE_THRESHOLD', 0.0, 1.0, row, resolution=0.05)
    
    def create_scrollable(self, parent):
        """Create scrollable frame for parameters"""
        canvas = tk.Canvas(parent, height=600)
        scrollbar = ttk.Scrollbar(parent, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        return scrollable_frame
    
    def add_slider(self, parent, label, params_obj, attr_name, min_val, max_val, row, resolution=1, step=1):
        """Add parameter slider with label and value display"""
        ttk.Label(parent, text=label, font=('Arial', 8)).grid(row=row, column=0, sticky=tk.W, padx=5, pady=2)
        
        current_val = getattr(params_obj, attr_name)
        var = tk.DoubleVar(value=current_val) if resolution < 1 else tk.IntVar(value=current_val)
        
        slider = ttk.Scale(parent, from_=min_val, to=max_val, variable=var, orient=tk.HORIZONTAL, length=200)
        slider.grid(row=row, column=1, padx=5)
        
        value_label = ttk.Label(parent, text=f"{current_val}", font=('Arial', 9, 'bold'), width=8)
        value_label.grid(row=row, column=2, padx=5)
        
        def update(val):
            new_val = round(float(val) / step) * step if step != 1 else float(val)
            if resolution >= 1:
                new_val = int(new_val)
            setattr(params_obj, attr_name, new_val)
            value_label.config(text=f"{new_val}")
        
        slider.configure(command=update)
    
    def update_hsv_range(self, range_idx, bound_idx, channel, value):
        """Update HSV range values"""
        self.params.HSV_RANGES[range_idx][bound_idx][channel] = value
    
    def start_detection(self):
        """Start detection loop"""
        if not self.is_running:
            self.is_running = True
            threading.Thread(target=self.detection_loop, daemon=True).start()
            self.update_display()
    
    def stop_detection(self):
        """Stop detection"""
        self.is_running = False
    
    def detection_loop(self):
        """Main detection processing loop"""
        fps_counter = 0
        fps_start = time.time()
        
        while self.is_running:
            try:
                # Generate test frame (replace with camera frame)
                test_frame = self.generate_test_frame()
                
                # Process frame
                results = self.detector.process_frame(test_frame)
                
                # Visualize
                vis_frame = self.detector.visualize_results(
                    results,
                    show_grid=self.show_grid.get(),
                    show_frequencies=self.show_frequencies.get(),
                    show_confidence=self.show_confidence.get()
                )
                
                # Queue for display
                if not self.frame_queue.full():
                    self.frame_queue.put_nowait((vis_frame, results))
                
                # Calculate FPS
                fps_counter += 1
                if time.time() - fps_start >= 1.0:
                    fps = fps_counter / (time.time() - fps_start)
                    fps_counter = 0
                    fps_start = time.time()
                    self.master.after(0, lambda: self.fps_label.config(text=f"FPS: {fps:.1f}"))
                
                time.sleep(0.033)  # ~30 FPS
            except Exception as e:
                print(f"Detection error: {e}")
                time.sleep(0.1)
    
    def update_display(self):
        """Update video display from queue"""
        if self.is_running:
            try:
                while not self.frame_queue.empty():
                    vis_frame, results = self.frame_queue.get_nowait()
                    self.display_frame(vis_frame)
                    
                    # Update status
                    self.detections_label.config(text=f"Detections: {len(results['detections'])}")
                    self.confirmed_label.config(text=f"Confirmed LEDs: {len(results['confirmed_leds'])}")
            except:
                pass
            
            self.master.after(30, self.update_display)
    
    def display_frame(self, frame):
        """Display frame on canvas"""
        if frame is None:
            return
        
        try:
            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Resize to fit canvas
            h, w = frame_rgb.shape[:2]
            canvas_w, canvas_h = 960, 720
            scale = min(canvas_w/w, canvas_h/h)
            new_w, new_h = int(w * scale), int(h * scale)
            
            resized = cv2.resize(frame_rgb, (new_w, new_h))
            
            # Convert to PhotoImage
            img = Image.fromarray(resized)
            photo = ImageTk.PhotoImage(img)
            
            # Display
            self.video_canvas.delete("all")
            self.video_canvas.create_image(canvas_w//2, canvas_h//2, image=photo)
            self.video_canvas.image = photo
        except Exception as e:
            print(f"Display error: {e}")
    
    def generate_test_frame(self):
        """Generate test frame with simulated LEDs (replace with camera)"""
        self.test_frame_count += 1
        
        # Create blank frame
        frame = np.random.randint(50, 100, (480, 640, 3), dtype=np.uint8)
        
        # Add simulated LEDs (flickering yellow dots)
        for i in range(3):
            x, y = 150 + i * 200, 200 + i * 50
            intensity = 200 + 55 * np.sin(2 * np.pi * 2 * self.test_frame_count / 30)  # 2 Hz flicker
            color = (0, int(intensity), int(intensity))  # Yellow in BGR
            cv2.circle(frame, (x, y), 15, color, -1)
        
        return frame

print("✅ Advanced GUI created")

✅ Advanced GUI created


In [32]:
# ===== CAMERA SERVICE & APPLICATION LAUNCHER =====

class CameraService:
    """Handle video input from RTSP, file, or webcam"""
    
    def __init__(self, source_type='test', source_path=None):
        self.source_type = source_type  # 'test', 'rtsp', 'file', 'webcam'
        self.source_path = source_path
        self.cap = None
        self.is_connected = False
        
    def connect(self):
        """Initialize video source"""
        try:
            if self.source_type == 'rtsp':
                self.cap = cv2.VideoCapture(self.source_path)
            elif self.source_type == 'file':
                self.cap = cv2.VideoCapture(self.source_path)
            elif self.source_type == 'webcam':
                self.cap = cv2.VideoCapture(0)
            elif self.source_type == 'test':
                self.is_connected = True
                return True
            
            if self.cap and self.cap.isOpened():
                self.is_connected = True
                return True
            return False
        except Exception as e:
            print(f"Camera connection failed: {e}")
            return False
    
    def get_frame(self):
        """Read next frame"""
        if self.source_type == 'test':
            return self.generate_test_frame()
        
        if self.cap and self.is_connected:
            ret, frame = self.cap.read()
            return frame if ret else None
        return None
    
    def generate_test_frame(self):
        """Generate test frame with simulated 2Hz LEDs"""
        frame_num = int(time.time() * 30) % 1000
        
        # Background
        frame = np.random.randint(40, 80, (480, 640, 3), dtype=np.uint8)
        
        # Add 3 simulated LEDs with 2Hz flicker
        led_positions = [(150, 200), (320, 180), (500, 250)]
        for x, y in led_positions:
            # 2Hz sine wave (60 frames per cycle at 30fps)
            intensity = 180 + 75 * np.sin(2 * np.pi * 2 * frame_num / 30)
            color = (0, int(intensity * 0.9), int(intensity))  # Yellow in BGR
            size = np.random.randint(12, 18)
            cv2.circle(frame, (x, y), size, color, -1)
            # Add slight glow
            cv2.circle(frame, (x, y), size + 5, color, 2)
        
        return frame
    
    def disconnect(self):
        """Release resources"""
        if self.cap:
            self.cap.release()
        self.is_connected = False


class LEDDetectionApp:
    """Main application launcher"""
    
    def __init__(self, camera_source='test', camera_path=None):
        self.root = tk.Tk()
        
        # Initialize detection system
        self.params = DetectionParams()
        self.detector = AdvancedLEDDetector(self.params)
        
        # Initialize camera
        self.camera = CameraService(camera_source, camera_path)
        if not self.camera.connect():
            print("⚠ Camera connection failed, using test mode")
            self.camera = CameraService('test')
            self.camera.connect()
        
        # Create GUI
        self.gui = AdvancedLEDDetectionGUI(self.root, self.detector)
        
        # Modify GUI to use camera service
        self.gui.generate_test_frame = self.camera.get_frame
        
        print("✅ LED Detection System initialized")
        print(f"   Camera: {camera_source}")
        print(f"   Target: 2Hz DC LEDs")
        print(f"   Range: 10-120 meters")
    
    def run(self):
        """Start application"""
        try:
            print("\n🚀 Starting GUI...")
            self.root.mainloop()
        except KeyboardInterrupt:
            print("\n⏹ Shutting down...")
        finally:
            self.camera.disconnect()


# ===== LAUNCH APPLICATION =====

def launch_detection_system(camera_type='test', camera_path=None):
    """
    Launch LED detection system
    
    Args:
        camera_type: 'test', 'rtsp', 'file', 'webcam'
        camera_path: Path for RTSP/file sources
    
    Examples:
        launch_detection_system('test')
        launch_detection_system('rtsp', 'rtsp://192.168.1.100:8080/video')
        launch_detection_system('file', 'video.mp4')
        launch_detection_system('webcam')
    """
    app = LEDDetectionApp(camera_type, camera_path)
    app.run()


print("✅ Application launcher ready")
print("\nTo start:")
print("  launch_detection_system('test')  # Test mode with simulated LEDs")
print("  launch_detection_system('rtsp', 'rtsp://your_camera_ip')  # Real camera")
print("  launch_detection_system('file', 'path/to/video.mp4')  # Video file")

✅ Application launcher ready

To start:
  launch_detection_system('test')  # Test mode with simulated LEDs
  launch_detection_system('rtsp', 'rtsp://your_camera_ip')  # Real camera
  launch_detection_system('file', 'path/to/video.mp4')  # Video file


In [33]:
# ===== LAUNCH THE APPLICATION =====

# To start the GUI with test LEDs (simulated):
launch_detection_system('test')

# For real camera (RTSP):
# launch_detection_system('rtsp', 'rtsp://192.168.1.100:8080/video')

# For video file:
# launch_detection_system('file', 'path/to/video.mp4')

# For webcam:
# launch_detection_system('webcam')

✅ LED Detection System initialized
   Camera: test
   Target: 2Hz DC LEDs
   Range: 10-120 meters

🚀 Starting GUI...
